## Answer questions

In [343]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [344]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR, MODELS_DIR, RAW_DATA_DIR

In [345]:
data = pd.read_csv(TRANSFORMED_DATA_DIR / 'transformed_data.csv')

In [346]:
data.shape

(4817, 22)

In [347]:
# Convert date columns to datetime
data['registration_date'] = pd.to_datetime(data['registration_date'])
data['sold_at'] = pd.to_datetime(data['sold_at'])

In [348]:
# Change data types from object to categorical
from src.data import convert_object_columns_to_category

data = convert_object_columns_to_category(data)

In [349]:
from src.data import get_train_test_data

In [350]:
import joblib

In [351]:
features = joblib.load(MODELS_DIR / 'features.pkl')
target = joblib.load(MODELS_DIR / 'target.pkl')

In [411]:
# Print the features
features

['mileage',
 'engine_power',
 'fuel',
 'paint_color',
 'car_type',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'age_in_months_when_sold',
 'month_sold_at',
 'season_sold_at',
 'model_initial']

In [352]:
X, y, X_train, X_test, y_train, y_test = get_train_test_data(data, features, target)

In [353]:
# Load the model
model = joblib.load(MODELS_DIR / 'model.pkl')

In [409]:
# Print the model
model

RandomizedSearchCV(cv=4,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=True,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=N...
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 0.2, 0.3],
                                        'max_delta_step': [0, 0.5, 1],
                                        'max_depth': [1, 2, 3, 4, 5, 6],
                                        'n_estimators': [1, 5, 10, 50, 100],
                                        'random_state': [42]},
                   random_state=0, scoring='neg_mean_squared_error', verbose=3)

### Q1

In [354]:
# Read feature importance df from models folder
feature_importance = joblib.load(MODELS_DIR / 'feature_importance_df.pkl')

In [355]:
feature_importance

,feature,importance
1,engine_power,0.300262
12,feature_8,0.145404
16,model_initial,0.124069
13,age_in_months_when_sold,0.113622
0,mileage,0.057269
9,feature_5,0.047976
5,feature_1,0.030138
3,paint_color,0.029260
8,feature_4,0.029180
4,car_type,0.018999


### Q2

As found during the data exploration phase:
- Hybrid and electrical cars are more expensive on average.
- Electrical cars average prices were stable from winter to summer, and were not sold in autumn.
- Diesel and petrol cars had similar average prices, although petrol cars had a drop in average prices starting summer 2018.
- The most expensive car type is, on average, suv, although coupe was most expensive at the start of the year and then dropped below suv also starting in summer.
- Coupe and convertible cars were, on average, more expensive in winter than in summer.
- Vans where more expensive, on average, in spring, summer, and autumn, than in winter.
- Subcompact had generally the lowest average prices.
- Paint color does not seem to generally determine or be associated with the average price, except for color green, which consistently had prices much lower than other colors. Maybe not very popular.
- Orange and white cars were sold for more, on average, during summer than during winter and spring.
- Red cars were the opposite, with lower average prices during summer than during winter and spring.

Find similar observations using estimated price instead of real price

In [356]:
pred = model.predict(X)

In [357]:
data_q2 = data.copy()

In [358]:
data_q2['price'] = pred

In [359]:
from src.plots import plot_avg_target_time_series_by_features

In [360]:
# Load car features
car_features = joblib.load(RAW_DATA_DIR / 'car_features.pkl')

In [361]:
plot_avg_target_time_series_by_features(data_q2, car_features)

In [362]:
# Load small cardinality features
small_cardinality_features = joblib.load(RAW_DATA_DIR / 'small_cardinality_features.pkl')

In [363]:
plot_avg_target_time_series_by_features(data_q2, small_cardinality_features)

In [364]:
from src.questions import ttest_mean_price_difference_between_groups_after_filter

In [365]:
grouping_column = 'season_sold_at'
group_1 = 'winter'
group_2 = 'summer'
for feature in small_cardinality_features:
    for feature_value in data_q2[feature].unique():
        t_stat, p_val = ttest_mean_price_difference_between_groups_after_filter(data_q2, feature, feature_value, grouping_column, group_1, group_2)
        print(f'Feature: {feature}, Feature Value: {feature_value}, T-stat: {t_stat:.4f}, P-value: {p_val:.4f}') 

Feature: fuel, Feature Value: diesel, T-stat: -0.9673, P-value: 0.3335
Feature: fuel, Feature Value: petrol, T-stat: 2.1616, P-value: 0.0339
Feature: fuel, Feature Value: hybrid_petrol, T-stat: nan, P-value: nan
Feature: fuel, Feature Value: electro, T-stat: -0.1037, P-value: 0.9342
Feature: car_type, Feature Value: convertible, T-stat: 1.0581, P-value: 0.3026
Feature: car_type, Feature Value: coupe, T-stat: 3.1474, P-value: 0.0030
Feature: car_type, Feature Value: estate, T-stat: 2.4725, P-value: 0.0136
Feature: car_type, Feature Value: hatchback, T-stat: -0.2044, P-value: 0.8381
Feature: car_type, Feature Value: sedan, T-stat: 4.2003, P-value: 0.0000
Feature: car_type, Feature Value: subcompact, T-stat: 0.5243, P-value: 0.6028
Feature: car_type, Feature Value: suv, T-stat: -1.3348, P-value: 0.1824
Feature: car_type, Feature Value: van, T-stat: -2.3284, P-value: 0.0325
Feature: paint_color, Feature Value: black, T-stat: 1.0497, P-value: 0.2942
Feature: paint_color, Feature Value: grey

### Q3

In [366]:
today_date = '3/1/2024'

In [367]:
data_q3 = data.copy()

In [368]:
# Convert date columns to datetime and mock the sold_at date as today's date
data_q3['registration_date'] = pd.to_datetime(data_q3['registration_date'])
data_q3['sold_at'] = today_date
data_q3['sold_at'] = pd.to_datetime(data_q3['sold_at'])

In [369]:
# Calculate mileage per month
data_q3['mileage_per_month'] = data_q3['mileage'] / data_q3['age_in_months_when_sold']

In [370]:
# Calculate age in month at today date and replace in data_q3
data_q3['age_in_months_when_sold'] = (data_q3['sold_at'].dt.to_period('M') - data_q3['registration_date'].dt.to_period('M')).apply(lambda x: x.n)

In [371]:
# Update estimated mileage at today date
data_q3['mileage'] = data_q3['age_in_months_when_sold'] * data_q3['mileage_per_month']

In [372]:
data_q3[['registration_date', 'sold_at', 'age_in_months_when_sold', 'mileage']].head()

,registration_date,sold_at,age_in_months_when_sold,mileage
0,2012-02-01,2024-03-01,145,286754.859155
1,2016-04-01,2024-03-01,95,60147.954545
2,2012-04-01,2024-03-01,143,374449.585714
3,2014-07-01,2024-03-01,116,345396.744186
4,2014-12-01,2024-03-01,111,269444.175000


In [373]:
X_q3, y_q3, X_train_q3, X_test_q3, y_train_q3, y_test_q3 = get_train_test_data(data_q3, features, target)

In [374]:
# Get estimated prices today
pred_q3 = model.predict(X_q3)

In [375]:
# Add 1 year to the age_in_months_when_sold
X_q3['age_in_months_when_sold'] = X_q3['age_in_months_when_sold'] + 12

In [376]:
# Add 1 year worth of mileage
X_q3['mileage'] = X_q3['mileage'] + 12 * data_q3['mileage_per_month']

In [377]:
# Get estimated prices 1 year later
pred_q3_one_year_later = model.predict(X_q3)

In [378]:
X_q3['price_today'] = pred_q3
X_q3['price_one_year_later'] = pred_q3_one_year_later
X_q3['loss'] = X_q3['price_today'] - X_q3['price_one_year_later'] 

In [379]:
# Indentify cars that are candidates for buying
candidate_cars = X_q3[(X_q3['price_today'] >= 20000) & (X_q3['loss'] <= 1000)]

In [380]:
# Show top 10 candidate cars with lowest loss
candidate_cars.sort_values('loss').head(10)

,mileage,engine_power,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,age_in_months_when_sold,month_sold_at,season_sold_at,model_initial,price_today,price_one_year_later,loss
90,50526.666667,170,hybrid_petrol,grey,coupe,True,True,False,False,False,False,False,True,110,4,spring,i,37873.863281,38197.425781,-323.562500
67,66996.268657,309,petrol,silver,coupe,True,True,False,False,True,True,True,True,150,4,spring,M,29584.013672,29584.013672,0.000000
2664,37186.666667,195,diesel,grey,sedan,True,True,True,False,False,True,True,True,100,2,winter,5,25716.244141,25517.945312,198.298828
3565,53503.157895,195,diesel,grey,sedan,True,True,True,True,False,True,True,True,97,9,autumn,5,26600.007812,26244.248047,355.759766
3910,9278.857143,155,diesel,black,suv,True,True,False,True,True,True,True,True,92,2,winter,X,23711.910156,23185.000000,526.910156
4615,103518.040816,180,diesel,black,suv,True,True,False,True,True,False,True,True,128,8,summer,X,21830.814453,20891.626953,939.187500


In [381]:
# Find the value counts for model_key in candidate_cars if exists. If not try model key initial.
try:
    model_related_value_counts = candidate_cars['model_key'].value_counts()
except KeyError:
    model_related_value_counts = candidate_cars['model_initial'].value_counts()

In [382]:
model_related_value_counts

model_initial
5    2
X    2
M    1
i    1
1    0
2    0
3    0
4    0
6    0
7    0
A    0
Z    0
Name: count, dtype: int64

In [383]:
# Describe numerical features of candidate cars
candidate_cars.describe()

,mileage,engine_power,age_in_months_when_sold,month_sold_at,price_today,price_one_year_later,loss
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,53501.609641,200.666667,112.833333,4.833333,27552.806641,27270.041016,282.765625
std,31351.008597,55.232840,22.220861,2.994439,5699.552246,6103.376465,436.015869
min,9278.857143,155.000000,92.000000,2.000000,21830.814453,20891.626953,-323.562500
25%,40521.666667,172.500000,97.750000,2.500000,24212.993652,23768.236328,49.574707
50%,52014.912281,187.500000,105.000000,4.000000,26158.125977,25881.096680,277.029297
75%,63622.990966,195.000000,123.500000,7.000000,28838.012207,28749.072266,484.122559
max,103518.040816,309.000000,150.000000,9.000000,37873.863281,38197.425781,939.187500


In [384]:
# Show the car with the lowest loss
car_index = candidate_cars.sort_values('loss').index[0]
data_q3.loc[car_index]

maker_key                                  BMW
model_key                                   i8
mileage                           45014.666667
engine_power                               170
registration_date          2016-01-01 00:00:00
fuel                             hybrid_petrol
paint_color                               grey
car_type                                 coupe
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                False
feature_7                                False
feature_8                                 True
price                                    95200
sold_at                    2024-03-01 00:00:00
age_in_months_when_sold                     98
month_sold_at                                4
season_sold_at                          spring
model_initial

In [385]:
data.loc[car_index]

maker_key                                  BMW
model_key                                   i8
mileage                                  12402
engine_power                               170
registration_date          2016-01-01 00:00:00
fuel                             hybrid_petrol
paint_color                               grey
car_type                                 coupe
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                False
feature_7                                False
feature_8                                 True
price                                    95200
sold_at                    2018-04-01 00:00:00
age_in_months_when_sold                     27
month_sold_at                                4
season_sold_at                          spring
model_initial

In [386]:
X_q3.loc[car_index]

mileage                     50526.666667
engine_power                         170
fuel                       hybrid_petrol
paint_color                         grey
car_type                           coupe
feature_1                           True
feature_2                           True
feature_3                          False
feature_4                          False
feature_5                          False
feature_6                          False
feature_7                          False
feature_8                           True
age_in_months_when_sold              110
month_sold_at                          4
season_sold_at                    spring
model_initial                          i
price_today                 37873.863281
price_one_year_later        38197.425781
loss                           -323.5625
Name: 90, dtype: object

### Q3 assumming today is a month after latest sold at date in the data

In [387]:
today_date = data['sold_at'].max()

In [388]:
data_q3 = data.copy()

In [389]:
# Convert date columns to datetime and mock the sold_at date as today's date
data_q3['registration_date'] = pd.to_datetime(data_q3['registration_date'])
data_q3['sold_at'] = today_date
data_q3['sold_at'] = pd.to_datetime(data_q3['sold_at'])

In [390]:
# Calculate mileage per month
data_q3['mileage_per_month'] = data_q3['mileage'] / data_q3['age_in_months_when_sold']

In [391]:
# Calculate age in month at today date and replace in data_q3
data_q3['age_in_months_when_sold'] = (data_q3['sold_at'].dt.to_period('M') - data_q3['registration_date'].dt.to_period('M')).apply(lambda x: x.n)

In [392]:
# Update estimated mileage at today date
data_q3['mileage'] = data_q3['age_in_months_when_sold'] * data_q3['mileage_per_month']

In [393]:
data_q3[['registration_date', 'sold_at', 'age_in_months_when_sold', 'mileage']].head()

,registration_date,sold_at,age_in_months_when_sold,mileage
0,2012-02-01,2018-09-01,79,156231.957746
1,2016-04-01,2018-09-01,29,18360.954545
2,2012-04-01,2018-09-01,77,201626.700000
3,2014-07-01,2018-09-01,50,148877.906977
4,2014-12-01,2018-09-01,45,109234.125000


In [394]:
X_q3, y_q3, X_train_q3, X_test_q3, y_train_q3, y_test_q3 = get_train_test_data(data_q3, features, target)

In [395]:
# Get estimated prices today
pred_q3 = model.predict(X_q3)

In [396]:
# Add 1 year to the age_in_months_when_sold
X_q3['age_in_months_when_sold'] = X_q3['age_in_months_when_sold'] + 12

In [397]:
# Add 1 year worth of mileage
X_q3['mileage'] = X_q3['mileage'] + 12 * data_q3['mileage_per_month']

In [398]:
# Get estimated prices 1 year later
pred_q3_one_year_later = model.predict(X_q3)

In [399]:
X_q3['price_today'] = pred_q3
X_q3['price_one_year_later'] = pred_q3_one_year_later
X_q3['loss'] = X_q3['price_today'] - X_q3['price_one_year_later'] 

In [400]:
# Indentify cars that are candidates for buying
candidate_cars = X_q3[(X_q3['price_today'] >= 20000) & (X_q3['loss'] <= 1000)]

In [401]:
# Show top 10 candidate cars with lowest loss
candidate_cars.sort_values('loss').head(10)

,mileage,engine_power,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,age_in_months_when_sold,month_sold_at,season_sold_at,model_initial,price_today,price_one_year_later,loss
4499,103077.391304,140,diesel,grey,suv,True,True,True,True,True,False,True,True,60,7,summer,X,31694.890625,56292.101562,-24597.210938
4020,72344.538462,190,diesel,blue,suv,True,True,False,True,True,True,True,True,67,6,summer,X,37198.785156,55122.914062,-17924.128906
3763,72054.818182,135,diesel,silver,suv,False,False,False,False,False,False,False,True,69,7,summer,X,22820.839844,39385.968750,-16565.128906
131,61018.431373,120,diesel,blue,coupe,True,True,False,False,True,True,True,True,65,7,summer,4,25132.906250,39843.000000,-14710.093750
123,60717.800000,120,diesel,blue,coupe,True,True,False,False,True,True,True,True,65,6,summer,4,25132.906250,39843.000000,-14710.093750
4285,61562.959184,135,diesel,silver,suv,True,True,False,False,False,False,False,True,65,5,spring,X,22913.343750,34067.160156,-11153.816406
4111,104025.034483,140,diesel,blue,suv,True,True,False,True,True,False,True,True,46,4,spring,X,30775.349609,41392.972656,-10617.623047
48,66621.020408,135,diesel,grey,coupe,False,False,False,False,False,False,False,True,65,5,spring,4,25843.203125,36457.617188,-10614.414062
105,45031.734694,135,petrol,grey,coupe,False,True,False,False,True,True,True,True,65,5,spring,4,26854.503906,37112.148438,-10257.644531
2125,71985.413793,135,diesel,brown,hatchback,True,True,False,False,True,False,True,True,74,5,spring,5,22247.978516,32397.851562,-10149.873047


In [403]:
# Find the value counts for model_key in candidate_cars if exists. If not try model key initial.
try:
    model_related_value_counts = candidate_cars['model_key'].value_counts()
except KeyError:
    model_related_value_counts = candidate_cars['model_initial'].value_counts()

In [404]:
model_related_value_counts

model_initial
X    38
5    26
4    21
2     7
M     4
3     3
6     3
7     3
i     2
1     0
A     0
Z     0
Name: count, dtype: int64

In [405]:
# Describe numerical features of candidate cars
candidate_cars.describe()

,mileage,engine_power,age_in_months_when_sold,month_sold_at,price_today,price_one_year_later,loss
count,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000
mean,104786.308524,159.102804,63.841121,5.177570,24493.468750,26336.054688,-1842.584106
std,76595.864215,43.980041,18.050845,1.941664,4666.877441,7318.672852,4775.637695
min,2622.285714,75.000000,26.000000,1.000000,20063.605469,19077.642578,-24597.210938
25%,58777.960035,135.000000,51.000000,4.000000,21195.285156,20591.787109,-1576.121094
50%,77818.742857,140.000000,65.000000,5.000000,23410.066406,23740.015625,327.398438
75%,113832.383181,180.000000,76.000000,7.000000,25633.867188,30758.102539,721.708984
max,347105.370370,280.000000,129.000000,9.000000,40571.250000,56292.101562,988.310547


In [406]:
# Show the car with the lowest loss
car_index = candidate_cars.sort_values('loss').index[0]
data_q3.loc[car_index]

maker_key                                  BMW
model_key                                   X3
mileage                           82461.913043
engine_power                               140
registration_date          2014-09-01 00:00:00
fuel                                    diesel
paint_color                               grey
car_type                                   suv
feature_1                                 True
feature_2                                 True
feature_3                                 True
feature_4                                 True
feature_5                                 True
feature_6                                False
feature_7                                 True
feature_8                                 True
price                                    25900
sold_at                    2018-09-01 00:00:00
age_in_months_when_sold                     48
month_sold_at                                7
season_sold_at                          summer
model_initial

In [407]:
data.loc[car_index]

maker_key                                  BMW
model_key                                   X3
mileage                                  79026
engine_power                               140
registration_date          2014-09-01 00:00:00
fuel                                    diesel
paint_color                               grey
car_type                                   suv
feature_1                                 True
feature_2                                 True
feature_3                                 True
feature_4                                 True
feature_5                                 True
feature_6                                False
feature_7                                 True
feature_8                                 True
price                                    25900
sold_at                    2018-07-01 00:00:00
age_in_months_when_sold                     46
month_sold_at                                7
season_sold_at                          summer
model_initial

In [408]:
X_q3.loc[car_index]

mileage                    103077.391304
engine_power                         140
fuel                              diesel
paint_color                         grey
car_type                             suv
feature_1                           True
feature_2                           True
feature_3                           True
feature_4                           True
feature_5                           True
feature_6                          False
feature_7                           True
feature_8                           True
age_in_months_when_sold               60
month_sold_at                          7
season_sold_at                    summer
model_initial                          X
price_today                 31694.890625
price_one_year_later        56292.101562
loss                       -24597.210938
Name: 4499, dtype: object

### Q4

Check training notebook

### Q5

Add other findings from data exploration